# Fragment 3

This fragment is able to interpret the following sentences from the previous fragment:
* "John runs and Mary doesn't run" as `(run john)∧¬(run mary)`
* "the dog runs" as `run dog`
* "John loves Mary" as `love john mary`
* "it is necessary that John runs" as `□(run john)`
* "it is not possible that the dog kills John" as `¬◇(kill dog john)`
* "it is not the case that John loves Mary" as `¬(love john mary)`

This fragment is able to interpret the following new sentences:
* "John has to run" as `□(run john)`
* "the dog is not allowed to kill John" as `¬◇(kill dog john)`
* "John has to be allowed to run" as `□◇(run john)`

## Parsing

In [1]:
abstract TestGrammar = {
    cat
        S;     -- Sentence
        SMod;  -- Sentence modifier
        Vi;    -- Intransitive verb
        Vt;    -- Transitive verb
        Vmod;  -- Modal verb
        VP;    -- Verb phrase
        Npr;   -- Proper noun
        N;     -- Common noun
        NP;    -- Noun phrase
        Conj;  -- Conjunction
    fun
        -- Positive sentence: "John runs"
        spos : NP -> VP -> S;
        -- Negative sentence: "John doesn't run"
        sneg : NP -> VP -> S;
        -- Connected sentence:
        -- "John runs and Mary doesn't run"
        sconj : S -> Conj -> S -> S;
        -- Positive modified sentence: "it is necessary that John runs"
        smodpos : SMod -> S -> S;
        -- Negative modified sentence: "it is not necessary that John runs"
        smodneg : SMod -> S -> S;
        
        -- Build a verb phrase from an intransitive verb: "run"
        vpvi : Vi -> VP;
        -- Build a verb phrase from a transitive verb and a noun phrase
        -- "love" + "Mary" -> "love Mary"
        vpvt : Vt -> NP -> VP;
        -- Apply a modal verb to an existing verb phrase
        -- "have to" + "run" -> "have to run"
        vpvmod : Vmod -> VP -> VP;
        
        -- Build a noun phrase from a proper noun: "John"
        npnpr : Npr -> NP;
        -- Build a noun phrase from a common noun
        -- "dog" -> "the dog"
        npn : N -> NP;
        
        -- Verbs
        run, jump, laugh, sing, howl, scream : Vi;
        read, poison, eat, like, love, loath, kick, see, understand, kill : Vt;
        haveto, beallowedto : Vmod;
        bepossible, benecessary, bethecase : SMod;
        -- Nouns
        prudence, ethel, chester, jo, bertie, fiona, berta, john, mary, peter : Npr;
        book, cake, katze, golfer, dog, lecturer, student, singer : N;
        -- Conjunctions
        and, or : Conj;
}

Defined TestGrammar

In [2]:
concrete TestGrammarEng of TestGrammar = {
    oper
        -- The type of a verb
        VType : Type = { base : Str ; pos : Str ; neg : Str };
    lincat
        S = Str;        -- Sentence
        SMod = VType;   -- Sentence modifier
        Vi = VType;     -- Intransitive verb
        Vt = VType;     -- Transitive verb
        Vmod = VType;   -- Modal verb
        VP = VType;     -- Verb phrase
        Npr = Str;      -- Proper noun
        N = Str;        -- Common noun
        NP = Str;       -- Noun phrase
        Conj = Str;     -- Conjunction
    oper
        -- Helper function to construct verb
        -- mkV "run" = { pos = "runs" ; neg = "doesn't run"}
        mkV = overload {
            mkV : Str -> VType =
                \v -> { base = v ; pos = v + "s" ; neg = "doesn't" ++ v };
            mkV : Str -> Str -> VType =
                \b,pos -> { base = b ; pos = pos ; neg = "doesn't" ++ b };
        };
        -- Helper function to construct be verb
        -- mkBeV "possible" = { pos = "is possible" ; neg = "is not possible"}
        mkBeV : Str -> VType =
            \s -> { base = "be" ++ s ; pos = "is" ++ s ; neg = "is not" ++ s };
            
        -- Helper function to append a string to verb
        vpconcat : VType -> Str -> VType = \v,s -> {
            base = v.base ++ s;
            pos = v.pos ++ s;
            neg = v.neg ++ s;
        };
    lin
        -- Positive sentence: "John runs"
        -- spos : NP -> VP -> S
        spos npr v = npr ++ v.pos;
        -- Negative sentence: "John doesn't run"
        -- sneg : NP -> VP -> S
        sneg npr v = npr ++ v.neg;
        -- Connected sentence:
        -- "John runs and Mary doesn't run"
        -- sconj : S -> Conj -> S -> S
        sconj s1 c s2 = s1 ++ c ++ s2;
        -- Positive modified sentence: "it is necessary that John runs"
        -- smodpos : SMod -> S -> S
        smodpos smod s = "it" ++ smod.pos ++ "that" ++ s;
        -- Negative modified sentence: "it is not necessary that John runs"
        -- smodneg : SMod -> S -> S
        smodneg smod s = "it" ++ smod.neg ++ "that" ++ s;
        
        -- Build a verb phrase from an intransitive verb: "run"
        -- vpvi : Vi -> VP
        vpvi v = v;
        -- Build a verb phrase from a transitive verb and a noun phrase
        -- "love" + "Mary" -> "love Mary"
        -- vpvt : Vt -> NP -> VP
        vpvt v np = vpconcat v np;
        -- Apply a modal verb to an existing verb phrase
        -- "have to" + "run" -> "have to run"
        -- vpvmod : Vmod -> VP -> VP
        vpvmod vmod vp = vpconcat vmod vp.base;
        
        -- Build a noun phrase from a proper noun: "John"
        -- npnpr : Npr -> NP
        npnpr npr = npr;
        -- Build a noun phrase from a common noun
        -- "dog" -> "the dog"
        -- npn : N -> NP
        npn n = "the" ++ n;
        
        -- Verbs
        run = mkV "run"; jump = mkV "jump"; laugh = mkV "laugh"; sing = mkV "sing"; howl = mkV "howl"; scream = mkV "scream";
        read = mkV "read"; poison = mkV "poison"; eat = mkV "eat"; like = mkV "like"; love = mkV "love"; loath = mkV "loath";
        kick = mkV "kick"; see = mkV "see"; understand = mkV "understand"; kill = mkV "kill";
        haveto = mkV "have to" "has to"; beallowedto = mkBeV "allowed to";
        bepossible = mkBeV "possible"; benecessary = mkBeV "necessary"; bethecase = mkBeV "the case";
        -- Nouns
        prudence = "Prudence"; ethel = "Ethel"; chester = "Chester"; jo = "Jo"; bertie = "Bertie"; fiona = "Fiona"; berta = "Berta";
        john = "John"; mary = "Mary"; peter = "Peter";
        book = "book"; cake = "cake"; katze = "cat"; golfer = "golfer"; dog = "dog"; lecturer = "lecturer"; student = "student";
        singer = "singer"; 
        -- Conjunctions
        and = "and"; or = "or";
}

### Parsing test cases (old)

In [3]:
parse "John runs"

spos (npnpr john) (vpvi run)

In [4]:
parse "John doesn't run"

sneg (npnpr john) (vpvi run)

In [5]:
parse "John runs and Mary doesn't run"

sconj (spos (npnpr john) (vpvi run)) and (sneg (npnpr mary) (vpvi run))

In [6]:
parse "John doesn't jump or Mary runs"

sconj (sneg (npnpr john) (vpvi jump)) or (spos (npnpr mary) (vpvi run))

In [7]:
parse "the dog runs"

spos (npn dog) (vpvi run)

In [8]:
parse "John loves Mary"

spos (npnpr john) (vpvt love (npnpr mary))

In [9]:
parse "John loves Mary and Mary doesn't love John"

sconj (spos (npnpr john) (vpvt love (npnpr mary))) and (sneg (npnpr mary) (vpvt love (npnpr john)))

In [10]:
parse "it is necessary that John runs"

smodpos benecessary (spos (npnpr john) (vpvi run))

In [11]:
parse "it is not possible that the dog kills John"

smodneg bepossible (spos (npn dog) (vpvt kill (npnpr john)))

In [12]:
parse "it is not the case that John loves Mary"

smodneg bethecase (spos (npnpr john) (vpvt love (npnpr mary)))

### Parsing test cases (new)

In [13]:
parse "John has to run"

spos (npnpr john) (vpvmod haveto (vpvi run))

In [14]:
parse "the dog is not allowed to kill John"

sneg (npn dog) (vpvmod beallowedto (vpvt kill (npnpr john)))

In [15]:
parse "John has to be allowed to run"

spos (npnpr john) (vpvmod haveto (vpvmod beallowedto (vpvi run)))

## Semantics construction

In [16]:
theory plnq : ur:?LF =
    proposition : type ❘ # o ❙
    individual : type ❘ # ι ❙
    
    not : o ⟶ o         ❘ # ¬ 1 prec 100 ❙
    and : o ⟶ o ⟶ o     ❘ # 1 ∧ 2 prec 90 ❙
    
    or : o ⟶ o ⟶ o      ❘ # 1 ∨ 2 prec 80
                        ❘ = [a,b] ¬ (¬ a ∧ ¬ b) ❙
❚

theory modalplnq : ur:?LF =
    include ?plnq ❙
    box : o ⟶ o     ❘ # □ 1 prec 100 ❙
    diamond : o ⟶ o ❘ # ◇ 1 prec 100 ❙
❚

theory plnqTest : ?modalplnq =
    // individuals ❙
    prudence : ι ❙ ethel : ι ❙ chester : ι ❙ jo : ι ❙ bertie : ι ❙ fiona : ι ❙ berta : ι ❙ john : ι ❙ mary : ι ❙ peter : ι ❙
    book : ι ❙ cake : ι ❙ cat : ι ❙ golfer : ι ❙ dog : ι ❙ lecturer : ι ❙ student : ι ❙ singer : ι ❙
    // verbs ❙
    run : ι ⟶ o ❙ jump : ι ⟶ o ❙ laugh : ι ⟶ o ❙ sing : ι ⟶ o ❙ howl : ι ⟶ o ❙ scream : ι ⟶ o ❙
    read : ι ⟶ ι ⟶ o ❙ poison : ι ⟶ ι ⟶ o ❙ eat : ι ⟶ ι ⟶ o ❙ like : ι ⟶ ι ⟶ o ❙ love : ι ⟶ ι ⟶ o ❙ loath : ι ⟶ ι ⟶ o ❙
    kick : ι ⟶ ι ⟶ o ❙ see : ι ⟶ ι ⟶ o ❙ understand : ι ⟶ ι ⟶ o ❙ kill : ι ⟶ ι ⟶ o ❙
❚

Created theory plnq

In [17]:
view semantics : http://mathhub.info/comma/jupyter/TestGrammar.gf?TestGrammar -> ?plnqTest =
    S = o ❙             // Sentence ❙
    SMod = o ⟶ o ❙      // Sentence modifier ❙
    Vi = ι ⟶ o ❙        // Intransitive verb ❙
    Vt = ι ⟶ ι ⟶ o ❙    // Transitive verb ❙
    Vmod = o ⟶ o ❙      // Modal verb ❙
    VP = ι ⟶ o ❙        // Verb phrase ❙
    Npr = ι ❙           // Proper noun ❙
    N = ι ❙             // Common noun, as a simplification modeled incorrectly
                        // as an individual ❙
    NP = ι ❙            // Noun phrase ❙
    Conj = o ⟶ o ⟶ o ❙  // Conjunction ❙
    
    // Positive sentence: "John runs"
    // spos : Npr -> V -> S ❙
    spos = [npr, v] v npr ❙
    // Negative sentence: "John doesn't run"
    // sneg : Npr -> V -> S ❙
    sneg = [npr, v] ¬(v npr) ❙
    // Connected sentence:
    // "John runs and Mary doesn't run"
    // sconj : S -> Conj -> S -> S ❙
    sconj = [s1, conj, s2] conj s1 s2 ❙
    // Positive modified sentence: "it is necessary that John runs"
    // smodpos : SMod -> S -> S ❙
    smodpos = [smod, s] smod s ❙
    // Negative modified sentence: "it is not necessary that John runs"
    // smodneg : SMod -> S -> S ❙
    smodneg = [smod, s] ¬(smod s) ❙
    
    // Build a verb phrase from an intransitive verb: "run"
    // vpvi : Vi -> VP ❙
    vpvi = [v] v ❙
    // Build a verb phrase from a transitive verb and a noun phrase
    // "love" + "Mary" -> "love Mary"
    // vpvt : Vt -> NP -> VP ❙
    vpvt = [vt, addressee] [actor] vt actor addressee ❙
    // Apply a modal verb to an existing verb phrase
    // "have to" + "run" -> "have to run"
    // vpvmod : Vmod -> VP -> VP ❙
    vpvmod = [vmod, vp] [i] vmod (vp i) ❙
        
    // Build a noun phrase from a proper noun: "John"
    // npnpr : Npr -> NP ❙
    npnpr = [npr] npr ❙
    // Build a noun phrase from a common noun
    // "dog" -> "the dog"
    // npn : N -> NP ❙
    npn = [n] n ❙
    
    // Verbs ❙
    run = run ❙ jump = jump ❙ laugh = laugh ❙ sing = sing ❙ howl = howl ❙ scream = scream ❙
    read = read ❙ poison = poison ❙ eat = eat ❙ like = like ❙ love = love ❙ loath = loath ❙ kick = kick ❙ see = see ❙ understand = understand ❙ kill = kill ❙
    haveto = box ❙ beallowedto = diamond ❙
    bepossible = diamond ❙ benecessary = box ❙ bethecase = [p] p ❙
    // Nouns ❙
    prudence = prudence ❙ ethel = ethel ❙ chester = chester ❙ jo = jo ❙ bertie = bertie ❙ fiona = fiona ❙ berta = berta ❙ john = john ❙ mary = mary ❙ peter = peter ❙
    book = book ❙ cake = cake ❙ katze = cat ❙ golfer = golfer ❙ dog = dog ❙ lecturer = lecturer ❙ student = student ❙ singer = singer ❙
    // Conjunctions ❙
    and = and ❙ or = or ❙
❚

Created view semantics

### Semantics construction test cases (old)

In [18]:
parse "John runs" | construct

run john

In [19]:
parse "John doesn't run" | construct

¬(run john)

In [20]:
parse "John runs and Mary doesn't run" | construct

(run john)∧¬(run mary)

In [21]:
parse "John doesn't jump or Mary runs" | construct

¬(jump john)∨(run mary)

In [22]:
parse "the dog runs" | construct

run dog

In [23]:
parse "John loves Mary" | construct

love john mary

In [24]:
parse "John loves Mary and Mary doesn't love John" | construct

(love john mary)∧¬(love mary john)

In [25]:
parse "it is necessary that John runs" | construct

□(run john)

In [26]:
parse "it is not possible that the dog kills John" | construct

¬◇(kill dog john)

In [27]:
parse "it is not the case that John loves Mary" | construct

¬(love john mary)

### Semantics construction test cases (new)

In [28]:
parse "John has to run" | construct

□(run john)

In [29]:
parse "the dog is not allowed to kill John" | construct

¬◇(kill dog john)

In [30]:
parse "John has to be allowed to run" | construct

□◇(run john)